# The DMCBH Open Science Initiative

The goal of our Open Science Initiative is to establish local support for, and a framework leading to, the transformation of the UBC Djavad Mowafaghian Centre for Brain Health into an Open Science Institute modelled after the pioneering example of the McGill Neurological Institute (MNI). Developing and adapting open science principles to suit the unique needs of our Centre necessitates collecting information on attitudes, behaviours and perceived norms, as they pertain to open science, from everyone in our research community - faculty, staff and students.  In complement to the focus groups and interviews we have been conducting with you, we have provided a script below to bridge the gap between conducting analysis in Caiman (an open-source tool for scalable calcium imaging data analysis) and working with an NWB file format. NWB (Neurodata Without Borders) is a data standard which enables neuroscientists to share and use analysis tools for neurophysiology data in hopes of breaking down barriers in data sharing.

<div>
<img src="https://pbs.twimg.com/profile_images/1133961932950532096/15M5Fvdy_400x400.png" width="200" height="200"/>
</div>

<html><head><meta content="text/html; charset=UTF-8" http-equiv="content-type"><style type="text/css">ol</style></head><body class="c5"><p class="c0 c4"><span class="c3"></span></p><p class="c2 title" id="h.rrbabt268i6e"><h1>CaImAn&rsquo;s Demo Pipeline</h1></p><p class="c0"><span class="c3">
This script follows closely the demo_pipeline.py script but uses the
Neurodata Without Borders (NWB) file format for loading the input and saving
the output. It is meant as an example on how to use NWB files with CaImAn.
authors: @agiovann and @epnev. The notebook has been adjusted a bit and has more descriptions on the steps of the analysis in order to better suit the needs of the centre.

    
Additional note: make sure caiman is activated prior to launching jupyter notebook to run this script. This can be done so by navigating to the caiman_data folder with the terminal and using <br> <br> `source activate caiman` <br> <br>Once you have done so, you can execute the code segments that follow in the script.

In [ ]:
import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

from datetime import datetime
from dateutil.tz import tzlocal

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.paths import caiman_datadir
bpl.output_notebook()

### Set up logger (optional)
You can log to a file using the filename parameter, or make the output more or less verbose by setting level to `logging.DEBUG`, `logging.INFO`, `logging.WARNING`, or `logging.ERROR`. 

In [ ]:
logging.basicConfig(format=
                    "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s]"\
                    "[%(process)d] %(message)s",
                    level=logging.WARNING)

### Select file(s) to be processed or download them if not present

In [ ]:
fnames = [os.path.join(caiman_datadir(), 'example_movies/cropped_testv2.nwb')]

The estimates save path can be same or different from raw data path. Please note that if you plan on using the Caiman GUI for further analysis once the nwb file is created, you need to load in the estimates nwb file.

In [ ]:
save_path = os.path.join(caiman_datadir(), 'example_movies/caiman_estimates_cropped_tifv2.nwb')

We will now set dataset dependent parameters below. fr is the imaging rate in frames per second and decay_time is the length of a typical transient in seconds.

In [ ]:
fr = 15.
decay_time = 0.4 

Now, we can load the file and save it in the NWB format (if it doesn't exist already). The download_demo function will download the tif file indicated and return the path to the file which will be stored in the caiman_data directory.

In [ ]:
if not os.path.exists(fnames[0]):
    fnames_orig = 'm2_00002_cropped.tif'  # filename to be processed
    if fnames_orig in ['m2_00002_cropped.tif', 'demoMovie.tif']:
        fnames_orig = [download_demo(fnames_orig)]
    orig_movie = cm.load(fnames_orig, fr=fr) #loads in dataset

The file can be now saved in an NWB format with additional information.

In [ ]:
orig_movie.save(fnames[0], sess_desc='test', identifier='demo 1',
        imaging_plane_description='single plane',
        emission_lambda=520.0, indicator='GCAMP6f',
        location='parietal cortex',
        experimenter='Marja Sepers', lab_name='Raymond Lab',
        institution='UBC',
        experiment_description='Experiment Description',
        session_id='Session 1',
        var_name_hdf5='TwoPhotonSeries')

# Motion Correction

We can now set up parameters for data and motion correction. Upon doing so, we pass all the params set as a dictionary. The goal of motion correction is to enable mapping the pixels to the same spatial positions.

Please note that there is a tradeoff between how non-rigid motion is vs the size of the patches.
Making the transformation peacewise rigid is more precise but will take a longer time for the analysis to complete.

In [ ]:
dxy = (2., 2.)  # spatial resolution in x and y in (um per pixel)
# note the lower than usual spatial resolution here
max_shift_um = (12., 12.)  # maximum shift in um
patch_motion_um = (100., 100.)  # patch size for non-rigid correction in um
pw_rigid = True       # flag to select rigid vs pw_rigid motion correction
max_shifts = [int(a/b) for a, b in zip(max_shift_um, dxy)] # maximum allowed rigid shift in pixels
strides = tuple([int(a/b) for a, b in zip(patch_motion_um, dxy)]) # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24) # overlap between patches (size of patch in pixels: strides+overlaps)
max_deviation_rigid = 3 # maximum deviation allowed for patch with respect to rigid shifts


mc_dict = {
    'fnames': fnames,
    'fr': fr,
    'decay_time': decay_time,
    'dxy': dxy,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': 'copy',
    #'var_name_hdf5': 'acquisition/TwoPhotonSeries'
    'var_name_hdf5': 'TwoPhotonSeries'
}

opts = params.CNMFParams(params_dict=mc_dict)

### Play the movie (optional)
Play the movie (optional). This will require loading the movie in memory which in general is not needed by the pipeline. Displaying the movie uses the OpenCV library. Press `q` to close the video panel.

Please note to set display_images to True if you wish to play the movie.

In [ ]:
display_images = True
if display_images:
    m_orig = cm.load_movie_chain(fnames, var_name_hdf5=opts.data['var_name_hdf5'])
    ds_ratio = 0.2
    moviehandle = m_orig.resize(1, 1, ds_ratio)
    moviehandle.play(q_max=99.5, fr=60, magnification=2, do_loop=True)

The next step is to start a cluster for parallel processing (this enables parallel motion correction by dividing the movie into chunks). This saves space and time.

In [ ]:
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

We are now ready to perform motion correction; the following line of code creates a motion correction object with the specified parameters. Note that the file is not loaded in memory.

In [ ]:
mc = MotionCorrect(fnames, dview=dview, var_name_hdf5=opts.data['var_name_hdf5'], **opts.get_group('motion'))

The next step runs a piecewise rigid motion correction using NoRMCorre. Note that the save_movie flag saves the motion corrected movie as a memory mapped file. 

A memory mapped file is an infrastructure which enables performing operations directly on the data stored on the hard drive. The default memory mapped file is created in order 'F' which is efficient for reading the file frame by frame.

In [ ]:
mc.motion_correct(save_movie=True)

The following lines of code enables you to compare the motion-corrected movie with the original. You can press q to exit. The left movie is the original and the right is the motion-corrected.

In [ ]:
if display_images:
    m_orig = cm.load_movie_chain(fnames, var_name_hdf5=opts.data['var_name_hdf5'])
    m_els = cm.load(mc.mmap_file)
    ds_ratio = 0.2
    moviehandle = cm.concatenate([m_orig.resize(1, 1, ds_ratio) - mc.min_mov*mc.nonneg_movie,
                                    m_els.resize(1, 1, ds_ratio)], axis=2)
    moviehandle.play(fr=60, q_max=99.5, magnification=2, do_loop=True)  

# Memory Mapping

In [ ]:
border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0

You can use the boundaries of the FOV if you used the copy option during motion correction (which is done so in the line of code above) but should be careful about the components near the boundaries. You can now memory map the file in order 'C'. This is efficient for reading the file pixel-wise.

In [ ]:
fname_new = cm.save_memmap(mc.mmap_file, base_name='memmap_', order='C',
                            border_to_0=border_to_0)

You can now load the file and easily read a subset of pixels.

In [ ]:
Yr, dims, T = cm.load_memmap(fname_new)
images = np.reshape(Yr.T, [T] + list(dims), order='F')

You can now restart the cluster to clean up memory.

In [ ]:
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

# Source Extraction and Deconvolution

Source extraction and deconvolution is done based off a batch analysis pipeline. The field of view will be divided in patches which will be sent to processors as tensors. The processors solve each patch (by defining the neurons) and put them together again. In other words, the chunks are processed and put together by merging the borders. The goal of source extraction  is to find the spatial footprint and signal of neurons. The goal of deconvolution is to find the noise to version of the signal with the auto-regressive model.

First, we set the required params for source extraction and deconvolution which will later be refined based on quality criteria.

Additional info on the params set:
<ul>
<li>p: defines how deconvolution is performed (p=1 is appropiate for fast indicators with slow frame rates such as 30 Hz with GCamp6F; p=2 is appropiate for slow indicators with fast frame rates such as GCamp6s)</li>
<li>gnb: neuropil (usually set to 1 but can also be set to 2 or 3 if the neuropil is complex)</li>   
<li>merge_thresh: can even go up to 0.9 to avoid putting together different neurons</li> 
</ul>

In [ ]:
p = 1                    # order of the autoregressive system 
gnb = 1                  # number of global background components 
merge_thr = 0.85         # merging threshold, max correlation allowed
rf = 15
# half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = 6          # amount of overlap between the patches in pixels
K = 4                    # number of components per patch (expected); usually overestimated
gSig = [4, 4]            # expected half size of neurons in pixels
# initialization method (if analyzing dendritic data using 'sparse_nmf')
method_init = 'greedy_roi'
ssub = 2                     # spatial subsampling during initialization
tsub = 2                     # temporal subsampling during intialization

In [ ]:
# parameters for component evaluation
opts_dict = {'fnames': fnames,
                'fr': fr,
                'nb': gnb,
                'rf': rf,
                'K': K,
                'gSig': gSig,
                'stride': stride_cnmf,
                'method_init': method_init,
                'rolling_sum': True,
                'merge_thr': merge_thr,
                'n_processes': n_processes,
                'only_init': True,
                'ssub': ssub,
                'tsub': tsub}

opts.change_params(params_dict=opts_dict);

We will now run CNMF on patches by first extracting spatial and temporal components on patches and combining them. Deconvolution is turned off for this step by setting p to 0.

In [ ]:
opts.change_params({'p': 0})
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm = cnm.fit(images)

You can now plot the contours of found components.

In [ ]:
Cn = cm.local_correlations(images, swap_dim=False)
Cn[np.isnan(Cn)] = 0
cnm.estimates.plot_contours(img=Cn)
plt.title('Contour plots of found components')

The results can be saved in a separate file.

In [ ]:
cnm.estimates.Cn = Cn
cnm.save(fname_new[:-4]+'hdf5')
cm.movie(Cn).save(fname_new[:-5]+'_Cn.tif')

To refine and perform deconvolution based on quality criteria, we rerun the seeded CNMF on accepted patches.

In [ ]:
cnm.params.change_params({'p': p})
cnm2 = cnm.refit(images, dview=dview)

The quality criteria used to rerun seeded CNMF are defined below:
<ul>
<li>min_SNR: how big the largest transient should be</li>
<li>rval_thr: how similar spatial component is with extraction vs FOV</li>
<li>cnn_thr: based on convolution network (supervised learning to differentiate neuron vs non-neuron)</li>
</ul>

We can now evaluate components in three ways:
1) The shape of each component must be correlated with the data <br>
2) A minimum peak SNR is required over the length of a transient <br>
3) Each shape passes a CNN based classifier <br>

In [ ]:
min_SNR = 2  # signal to noise ratio for accepting a component
rval_thr = 0.85  # space correlation threshold for accepting a component
cnn_thr = 0.99  # threshold for CNN based classifier
cnn_lowest = 0.1 # neurons with cnn probability lower than this value are rejected

cnm2.params.set('quality', {'decay_time': decay_time,
                            'min_SNR': min_SNR,
                            'rval_thr': rval_thr,
                            'use_cnn': True,
                            'min_cnn_thr': cnn_thr,
                            'cnn_lowest': cnn_lowest});
cnm2.estimates.evaluate_components(images, cnm2.params, dview=dview)
cnm2.estimates.Cn = Cn
cnm2.save(fname_new[:-4] + 'hdf5')

You can now plot components after refinement. The left plot is that of the components with good spatial footprint.

In [ ]:
cnm2.estimates.plot_contours(img=Cn, idx=cnm2.estimates.idx_components)

The code segment below enables you to view accepted and rejected traces. Please note that the first line enables who to view accepted components and the second line is for rejected components.

In [ ]:
if display_images:
    cnm2.estimates.view_components(images, img=Cn,
                                    idx=cnm2.estimates.idx_components)
    cnm2.estimates.view_components(images, img=Cn,
                                    idx=cnm2.estimates.idx_components_bad)

You can update the the object with selected components:

In [ ]:
cnm2.estimates.select_components(use_object=True)

To extract dF/F values and show final traces:

In [ ]:
cnm2.estimates.detrend_df_f(quantileMin=8, frames_window=250)
cnm2.estimates.view_components(img=Cn)

You can also now reconstruct denoised movie. The movie on the left is the raw movie; the one in the middle is the reconstructed movie; the right one is the residual movie. There should be clear spatial footprint in the middle and almost no components in the right one.

In [ ]:
if display_images:
    cnm2.estimates.play_movie(images, q_max=99.9, gain_res=2,
                                magnification=2,
                                bpx=border_to_0,
                                include_bck=False) #enables you to include background

Now, you can stop the cluster and clean up log files.

In [ ]:
cm.stop_server(dview=dview)
log_files = glob.glob('*_LOG_*')
for log_file in log_files:
    os.remove(log_file)

Next, save the files in the original NWB file.

In [ ]:
cnm2.estimates.save_NWB(save_path, imaging_rate=fr, session_start_time=datetime.now(tzlocal()),
                        raw_data_file=fnames[0])

You can continue to perform further analysis on this file by running the caiman.gui and loading the file. Y For example, if you are interested in df/F values, you can print them by executing the following line of code below. 

# Steps for Further Analysis

You can continue to view specific estimates that you are further interested in analyzing and make plots (such as dF/F traces).

In [ ]:
cnm2.estimates.F_dff

In [ ]:
plt.imshow(cnm2.estimates.F_dff[:,0:50])

In [ ]:
plt.plot(cnm2.estimates.F_dff[0,:])

If you would like to use the dF/F data for further analysis in other script, you can store the data as a variable for later use.

In [ ]:
dFOverFv2 = cnm2.estimates.F_dff

In [ ]:
%store dFOverFv2

## Notes on Using the Caiman Gui:

You can load the NWB file created by running this script in the GUI. You can select the neurons and view their signals as well as the background of the video. You can alter any of the thresholds as you see fit. On the left bottom, you can change the spatial footprint of the neuron selected as well as the contrast of the background. On the right bottom, you can manually select the neurons that you want and add group as the accepted ones. You can also remove a single neuron from the accepted components which would move it to the rejected ones. You can save the changes to the NWB file or create an HDF5 file.